In [ ]:
# fold
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from time import time
import os
import tensorflow as tf
from keras import applications
from keras.utils import multi_gpu_model
from sklearn.metrics import roc_curve

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.visible_device_list = "2,3"
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

# Globals
VERBOSE = 0
ARCHITECTURE = 0
NORMALISE = 1
# Class 0 = backgrounds
CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/normal/*'
CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/malignant/*'

# Calcs
#CLASSDIR_0 = '/user/HS204/wm0015/student/allCalcs/0/*'
#CLASSDIR_1 = '/user/HS204/wm0015/student/allCalcs/1/*'
MODEL_SAVE = '/vol/vssp/cvpwrkspc01/scratch/wm0015/models/best_model.h5'
BATCH_SIZE = 100
EPOCHS = 5000
#INPUT_SHAPE = [256, 256, 3]
INPUT_SHAPE = [429, 429, 1]
#INPUT_SHAPE = [385, 385, 3]
FOLDS = 5
PATIENCE = 500

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + '_' + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + '_' + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + '_' + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x

def fourCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    #model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model
    
def fiveCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model
    
def bigCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(64, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(124, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(256, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(512, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model

def vgg():
    vggModel = applications.VGG19(weights = 'imagenet', include_top=False, input_shape = INPUT_SHAPE)
    # Add custom final layer
    model = vggModel.output
    model = Flatten()(model)
    model = Dense(2, activation='softmax')(model)
    model = keras.models.Model(inputs=vggModel.input, outputs=model)
    return model

def tlVGG(train_data, val_data):
    print('Compute bottleneck features...')
    vggModel = applications.VGG19(weights = "imagenet", include_top=False, input_shape = INPUT_SHAPE)
    # Freeze all layers
    for layer in vggModel.layers:
        layer.trainable = False
    # Add custom final layer
    bNModel = vggModel.output
    bNModel = Flatten()(bNModel)
    final_model = keras.models.Model(inputs=vggModel.input, outputs=bNModel)
    train_bNFeatures = {'img': 0, 'label': train_data['label']}
    val_bNFeatures = {'img': 0, 'label': val_data['label']}
    train_bNFeatures['img'] = final_model.predict(train_data['img'], batch_size=16)
    val_bNFeatures['img'] = final_model.predict(val_data['img'], batch_size=16)
    #Undo one hot - ROC does not work with onehot
    val_bNFeatures.update({'labelIndex': np.where(val_data['label']==1)[1]})
    print('train_bNFeatures[img].shape = ', train_bNFeatures['img'].shape)
    print('val_bNFeatures[img].shape = ', val_bNFeatures['img'].shape)

    print('Train head...')
    head = Sequential()
    #head.add(Dense(32, input_dim=train_bNFeatures['img'].shape[1], activation='relu'))
    head.add(Dense(2, activation='softmax'))    
    return head, train_bNFeatures, val_bNFeatures

    # Why oh why are they in a directory structure like this
def getPremFiles(imgPath, dataSpecs):
    import pydicom
    from fnmatch import fnmatch
    # First get all the 6mm lesions
    fileList = []
    for path, subdirs, files in os.walk(imgPath):
        for name in files:
            if fnmatch(name, '2D_dim2d.dcm'):
                fileList.append(os.path.join(path, name))
    # I can't remember why I thought dataSpecs was a good idea
    # I suppose this means that class 0 needs to be loaded in first
    dataSpecs['classLength'].append(len(fileList))
    
    # Load the files from filelist into an array
    print(len(fileList), ' Files found')
    print('Loading images...')
    dicomImg = np.asarray([])
    count = 0
    for f in fileList:
        dicomImg = np.append(dicomImg, pydicom.dcmread(f).pixel_array)
        count += 1
        print(count, '/', len(fileList))
    return dicomImg

    
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    print('TensorFlow version: ', tf.__version__)
    print('\nLoad images...')
    
    # Get images and labels
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    
    # Comment out for prem images
#     data['img'] = np.concatenate((
#             get_images(CLASSDIR_0, dataSpecs), # Class 0 (backgrounds)
#             get_images(CLASSDIR_1, dataSpecs) # Class 1 
#     ))      

    past1 = get_images(CLASSDIR_0, dataSpecs)
    past2 = get_images(CLASSDIR_1, dataSpecs)
    pastConcat = np.concatenate((past1, past2))


    # Prem images
    prem1 = getPremFiles('/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/Segments', dataSpecs)
    prem2 = getPremFiles('/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/2D/6mm', dataSpecs)
    prem1 = prem1.reshape((-1, 429, 429, 1))
    prem2 = prem2.reshape((-1, 429, 429, 1))
#     print('prem1.shape: ', prem1.shape)
#     print('prem2.shape: ', prem2.shape)
#     print('past1.shape: ', past1.shape)
#     print('past2.shape: ', past2.shape)
#     print('pastConcat.shape: ', pastConcat.shape)
#     print('prem1[0].shape: ', prem1[0].shape)
#     print('prem2[0].shape: ', prem2[0].shape)
    data['img'] = np.concatenate((prem1, prem2))

    # Normalise
    data['img'] = data['img']/NORMALISE   
    # Print image    
#     img_calc = data['img']   
#     plt.imshow(img_calc[0], cmap='gray')
#     plt.show()
    
    # Create one hot labels
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    if 1==0:
        data['img'] = data['img'][:,:,:,0]
        data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
        print('new data shape = ', data['img'].shape)
    
    valStats = []
    for crossVal in range(FOLDS):

        # Shuffle data
        seed = 33
        np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0])    
        data['img'] = data['img'][shuffleMask, :, :, :]
        data['label'] = data['label'][shuffleMask, :]

        # Split traing and validation data        
        splitRatio = 0.9
        splitPoint = math.floor(data['img'].shape[0]*splitRatio)
        train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
        val_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
        #Undo one hot - ROC does not work with onehot
        val_data.update({'labelIndex': np.where(val_data['label']==1)[1]})

        

        #model, train_data, val_data = tlVGG(train_data, val_data)
        model = fiveCNN()
        if ARCHITECTURE != 0:
            model.summary()  
        model = multi_gpu_model(model, gpus=2)
        
        sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
        model.compile(loss='binary_crossentropy',
                    optimizer=adam,
                     metrics=['accuracy'])
        tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)

        # Data augmentation settings
        from keras.preprocessing.image import ImageDataGenerator
        trainDatagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True)
        trainDatagen.fit(train_data['img'])
        valDatagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            #rotation_range=20,
            #width_shift_range=0.2,
            #height_shift_range=0.2,
            #horizontal_flip=True
            )
        valDatagen.fit(val_data['img'])
        
#         # Normalise myself
#         mean_data = np.mean(train_data['img'])
#         std_data = np.std(train_data['img'])
#         train_data['img'] = (train_data['img']-mean_data)/std_data
#         val_data['img'] = (val_data['img']-mean_data/std_data)
        
#        # Train, with data augmentation
#         print('Train...')
#         model.fit_generator(trainDatagen.flow(train_data['img'], train_data['label'], batch_size=32),
#                     steps_per_epoch=len(train_data['img']) / 32, epochs=EPOCHS, verbose=VERBOSE,
#                            callbacks=[
#                                TrainValTensorBoard(write_graph=False),
#                                EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1),
#                                ModelCheckpoint(filepath=MODEL_SAVE, monitor='val_accuracy', save_best_only=True)],
#                            validation_data=valDatagen.flow(x = val_data['img'], y = val_data['label']))
        

        
        
        
        # Train
        print('Train...')
        model.fit(train_data['img'], train_data['label'], 
                batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE,
                callbacks=[
                    TrainValTensorBoard(write_graph=False),
                    EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1),
                    ModelCheckpoint(filepath=MODEL_SAVE, monitor='val_accuracy', save_best_only=True)],
                validation_data=(val_data['img'], val_data['label']))
        
        # Restore best model
        model.load_weights(MODEL_SAVE)
        
        # Produce ROC curve
        
        y_pred_keras = model.predict(val_data['img'])  
        y_true = val_data['labelIndex']
        y_score = y_pred_keras[:,1]
        fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true, y_score)
        from sklearn.metrics import auc
        auc_keras = auc(fpr_keras, tpr_keras)
        plt.figure(1)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')
        plt.show()


        # Evaluate
        score = model.evaluate(val_data['img'], val_data['label'], verbose=0)
        valStats.append(score)
        print('The score is......\n', score)
        
  

    valStats = np.asarray(valStats)
    print('Validations: \n', valStats[:, 1])
    print('Average loss: ', sum(valStats[:, 0])/FOLDS)
    print('Average validation: ', sum(valStats[:, 1])/FOLDS)
    
if __name__ == "__main__":
    main()

keras version:  2.2.0
TensorFlow version:  1.7.1

Load images...
1176  Files found
Loading images...
1 / 1176
2 / 1176
3 / 1176
4 / 1176
5 / 1176
6 / 1176
7 / 1176
8 / 1176
9 / 1176
10 / 1176
11 / 1176
12 / 1176
13 / 1176
14 / 1176
15 / 1176
16 / 1176
17 / 1176
18 / 1176
19 / 1176
20 / 1176
21 / 1176
22 / 1176
23 / 1176
24 / 1176
25 / 1176
26 / 1176
27 / 1176
28 / 1176
29 / 1176
30 / 1176
31 / 1176
32 / 1176
33 / 1176
34 / 1176
35 / 1176
36 / 1176
37 / 1176
38 / 1176
39 / 1176
40 / 1176
41 / 1176
42 / 1176
43 / 1176
44 / 1176
45 / 1176
46 / 1176
47 / 1176
48 / 1176
49 / 1176
50 / 1176
51 / 1176
52 / 1176
53 / 1176
54 / 1176
55 / 1176
56 / 1176
57 / 1176
58 / 1176
59 / 1176
60 / 1176
61 / 1176
62 / 1176
63 / 1176
64 / 1176
65 / 1176
66 / 1176
67 / 1176
68 / 1176
69 / 1176
70 / 1176
71 / 1176
72 / 1176
73 / 1176
74 / 1176
75 / 1176
76 / 1176
77 / 1176
78 / 1176
79 / 1176
80 / 1176
81 / 1176
82 / 1176
83 / 1176
84 / 1176
85 / 1176
86 / 1176
87 / 1176
88 / 1176
89 / 1176
90 / 1176
91 / 117

747 / 1176
748 / 1176
749 / 1176
750 / 1176
751 / 1176
752 / 1176
753 / 1176
754 / 1176
755 / 1176
756 / 1176
757 / 1176
758 / 1176
759 / 1176
760 / 1176
761 / 1176
762 / 1176
763 / 1176
764 / 1176
765 / 1176
766 / 1176
767 / 1176
768 / 1176
769 / 1176
770 / 1176
771 / 1176
772 / 1176
773 / 1176
774 / 1176
775 / 1176
776 / 1176
777 / 1176
778 / 1176
779 / 1176
780 / 1176
781 / 1176
782 / 1176
783 / 1176
784 / 1176
785 / 1176
786 / 1176
787 / 1176
788 / 1176
789 / 1176
790 / 1176
791 / 1176
792 / 1176
793 / 1176
794 / 1176
795 / 1176
796 / 1176
797 / 1176
798 / 1176
799 / 1176
800 / 1176
801 / 1176
802 / 1176
803 / 1176
804 / 1176
805 / 1176
806 / 1176
807 / 1176
808 / 1176
809 / 1176
810 / 1176
811 / 1176
812 / 1176
813 / 1176
814 / 1176
815 / 1176
816 / 1176
817 / 1176
818 / 1176
819 / 1176
820 / 1176
821 / 1176
822 / 1176
823 / 1176
824 / 1176
825 / 1176
826 / 1176
827 / 1176
828 / 1176
829 / 1176
830 / 1176
831 / 1176
832 / 1176
833 / 1176
834 / 1176
835 / 1176
836 / 1176
837 / 1176

337 / 531
338 / 531
339 / 531
340 / 531
341 / 531
342 / 531
343 / 531
344 / 531
345 / 531
346 / 531
347 / 531
348 / 531
349 / 531
350 / 531
351 / 531
352 / 531
353 / 531
354 / 531
355 / 531
356 / 531
357 / 531
358 / 531
359 / 531
360 / 531
361 / 531
362 / 531
363 / 531
364 / 531
365 / 531
366 / 531
367 / 531
368 / 531
369 / 531
370 / 531
371 / 531
372 / 531
373 / 531
374 / 531
375 / 531
376 / 531
377 / 531
378 / 531
379 / 531
380 / 531
381 / 531
382 / 531
383 / 531
384 / 531
385 / 531
386 / 531
387 / 531
388 / 531
389 / 531
390 / 531
391 / 531
392 / 531
393 / 531
394 / 531
395 / 531
396 / 531
397 / 531
398 / 531
399 / 531
400 / 531
401 / 531
402 / 531
403 / 531
404 / 531
405 / 531
406 / 531
407 / 531
408 / 531
409 / 531
410 / 531
411 / 531
412 / 531
413 / 531
414 / 531
415 / 531
416 / 531
417 / 531
418 / 531
419 / 531
420 / 531
421 / 531
422 / 531
423 / 531
424 / 531
425 / 531
426 / 531
427 / 531
428 / 531
429 / 531
430 / 531
431 / 531
432 / 531
433 / 531
434 / 531
435 / 531
436 / 531


/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/callbacks.py:432: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


In [5]:
print('hello world')

hello world


In [6]:
def test():
    return 1, 2, 3

x, y, z = test()
print(x)
print(y)dd
print(z)

1
2
3


In [35]:
x = [[6.5308382717179665, 0.5925925925925926], [0.16470135086112553, 0.9444444444444444], [0.15422181794304907, 0.9444444444444444], [0.19875195217721256, 0.9382716049382716]] 
x = np.asarray(x)
print(x, '\n')
print(x[0][0])
print(x[:, 0])
print(x.shape)

[[6.53083827 0.59259259]
 [0.16470135 0.94444444]
 [0.15422182 0.94444444]
 [0.19875195 0.9382716 ]] 

6.5308382717179665
[6.53083827 0.16470135 0.15422182 0.19875195]
(4, 2)
